<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex8-Efficient_Frontier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Foundations of Investment Analysis***, Bates, Boyer, and Fletcher

# Example 8: The Efficient Frontier



### Imports and Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.optimize import minimize

### Define Inputs
In the code cell below we define the covariance matrix and the vector of expected returns.  The default is for the case of three assets, but these can be for any $N$ assets, provided that the `covariance_matrix` is $N\times N$ and the array of `expected_returns` is $N \times 1$.   

In [ ]:
covariance_matrix = np.array([[0.0400, -0.0200, 0.0250], [-0.0200, 0.0900, 0.0400], [0.0250, 0.0400, 0.2500]])
expected_returns = np.array([[0.12], [0.06], [0.08]])
rf = 0.05
# check dimensions of inputs
print("Dimension of Covariance Matrix:",covariance_matrix.shape)
print("Dimension of Expected Returns:", expected_returns.shape)

Dimension of Covariance Matrix: (3, 3)
Dimension of Expected Returns: (3, 1)


### Define Functions to Determine Portfolios on EFRS
In this next code cell, we create two functions to use in determining portfolios on the EFRS.

`portfolio_volatility`  
inputs:  
(1) a vector of weights, and (2) the covariance matrix.  The output of this function is the volatility of the portfolio using Essential Math Fact #5.

`EFRS_portfolio`: inputs are (1) a single target portfolio expected return, (2) the vector of expected returns for individual assets, (3) the covaraince matrix. The    

In [ ]:
def portfolio_volatility(weights: np.array, covariance_matrix: np.array):
    return np.sqrt(weights.T @ covariance_matrix @ weights)


def EFRS_portfolio(target_return, expected_returns, covariance_matrix):
    constraints = (
        {"type": "eq", "fun": lambda w: w.T @ expected_returns - target_return}, {"type": "eq", "fun": lambda w: np.sum(w) - 1},)

    initial_weights = np.ones(3) / 3

    result = minimize(
        fun=portfolio_volatility,
        x0=initial_weights,
        args=(covariance_matrix),
        method="SLSQP",
        constraints=constraints,
        )

    return result.x

In [ ]:
# Target returns
start = 0.5 * min(expected_returns)
end = 2 * max(expected_returns)
target_returns = np.linspace(start, end, num=100)

mve_portfolios = [mve_portfolio(target, expected_returns, covariance_matrix) for target in target_returns]